In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import imageio
import decord
from hmr4d.utils.wis3d_utils import make_wis3d
from hmr4d.utils.wis3d_utils import add_joints22_motion_as_lines
from hmr4d.dataset.rich.rich_utils import get_w2az_sahmr, parse_seqname_info
from hmr4d.utils.geo_transform import apply_T_on_points

In [2]:
video_length = 779
clip_length = min(video_length, (120 - 1) * 4 + 1)
start_idx   = np.random.randint(0, video_length - clip_length)
batch_index = np.linspace(start_idx, start_idx + clip_length - 1, 120, dtype=int)
print(batch_index)

[291 295 299 303 307 311 315 319 323 327 331 335 339 343 347 351 355 359
 363 367 371 375 379 383 387 391 395 399 403 407 411 415 419 423 427 431
 435 439 443 447 451 455 459 463 467 471 475 479 483 487 491 495 499 503
 507 511 515 519 523 527 531 535 539 543 547 551 555 559 563 567 571 575
 579 583 587 591 595 599 603 607 611 615 619 623 627 631 635 639 643 647
 651 655 659 663 667 671 675 679 683 687 691 695 699 703 707 711 715 719
 723 727 731 735 739 743 747 751 755 759 763 767]


In [5]:
import decord
import time

decord_readout = "torch"  # ["torch", "native"]
decord.bridge.set_bridge(decord_readout)
video_reader = decord.VideoReader("inputs/RICH/hmr4d_support/video/test/Gym_010_cooking1/cam_01/video.mp4")

for i in range(11):
    if i == 1:
        start = time.time()
    if decord_readout == "torch":
        pixel_values = video_reader.get_batch(batch_index).permute(0, 3, 1, 2).contiguous()
        pixel_values_np = pixel_values.numpy()
    else:
        pixel_values = torch.from_numpy(video_reader.get_batch(batch_index).asnumpy()).permute(0, 3, 1, 2).contiguous()
    if i == 10:
        end = time.time()
        print((end - start) / 10)

print('===')

for i in range(11):
    if i == 1:
        start = time.time()
    if decord_readout == "torch":
        px = []
        for f in batch_index:
            pixel_values = video_reader[f].permute(2, 0, 1).contiguous()
            pixel_values_np = pixel_values.numpy()
            px.append(pixel_values_np)
        px = np.stack(px)
    else:
        pixel_values = torch.from_numpy(video_reader.get_batch(batch_index).asnumpy()).permute(0, 3, 1, 2).contiguous()
    if i == 10:
        end = time.time()
        print((end - start) / 10)

0.7969031572341919
===
1.2989567518234253
